# 3D Information Processing
Obtain a 3D shape from images taken by multiple cameras

In [1]:
import pandas as pd
from camera import Camera
from visualize import plot_calibration_points

Load calibration points for each images

In [2]:
points_1 = pd.read_csv("points/points_1.csv")
points_2 = pd.read_csv("points/points_2.csv")
points_3 = pd.read_csv("points/points_3.csv")

Plot calibration points

In [3]:
plot_calibration_points("data/1.JPG", "data/1_plotted.JPG", points_1)
plot_calibration_points("data/2.JPG", "data/2_plotted.JPG", points_2)
plot_calibration_points("data/3.JPG", "data/3_plotted.JPG", points_3)

### 1. Obtain perspective projection matrix

Calibrate cameras from calibration points

In [4]:
c1 = Camera(points_1)
c2 = Camera(points_2)
c3 = Camera(points_3)

In [5]:
c1.calibrate()
c2.calibrate()
c3.calibrate()

Calibrate camera...
Perspective Projection Matrix
[[-4.00609975e+01  2.65399990e+01 -2.45974706e+01  1.57947571e+03]
 [ 8.95936948e+00  7.63276967e-01 -5.16027868e+01  1.11046870e+03]
 [-1.38863601e-02 -6.90024605e-03 -1.25798052e-02  1.00000000e+00]]
Calibrate camera...
Perspective Projection Matrix
[[-4.67327233e+01  1.97686309e+01 -2.06704973e+01  1.58328332e+03]
 [ 1.64934612e+00 -9.27733772e-01 -5.14408669e+01  1.06308325e+03]
 [-1.45062690e-02 -8.80332269e-03 -1.04645889e-02  1.00000000e+00]]
Calibrate camera...
Perspective Projection Matrix
[[-4.57784751e+01  1.68717312e+01 -2.64678054e+01  1.66781077e+03]
 [ 1.17659335e+01  7.14988008e+00 -4.85031209e+01  8.86641441e+02]
 [-1.14629726e-02 -8.62416880e-03 -1.34705241e-02  1.00000000e+00]]


Validate perspective projection matrix by other points

In [6]:
points_valid = pd.read_csv("points/points_1_valid.csv")
pred_u = []
pred_v = []
for row in points_valid.itertuples():
    u_, v_ = c1.perspective_project(row[3], row[4], row[5])
    pred_u += [u_]
    pred_v += [v_]

predict = pd.DataFrame({"u": pred_u, "v": pred_v,
                        "x": points_valid["x"],
                        "y": points_valid["y"],
                        "z": points_valid["z"]})

predict["u"] = predict["u"].astype(int)
predict["v"] = predict["v"].astype(int)

plot_calibration_points("data/1.JPG", "data/1_pred.JPG", points_valid)
plot_calibration_points("data/1_pred.JPG", "data/1_pred.JPG",
                        predict, color=(0, 0, 230))

Re-project calibration points

In [7]:
c1.re_project()

plot_calibration_points("data/1.JPG", "data/1_reproject.JPG",
                        c1.point_for_calibration)
plot_calibration_points("data/1_reproject.JPG", "data/1_reproject.JPG",
                        c1.points_of_reprojection,
                        color=(0, 0, 230))
reprojection_error = c1.re_projection_error()
reprojection_error

98.77887947958108

### 2. Obtain 3D points from 2 images
